In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
import numpy as np

In [3]:
from collections import Counter

In [4]:
#import data in csv file
review_static = pd.read_csv('Amazon_Unlocked_Mobile.csv')
review = review_static.copy(deep=True)

In [5]:
review.head(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0
5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I already had a phone with problems... I know ...,1.0
6,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0
7,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,"Phone looks good but wouldn't stay charged, ha...",0.0
8,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0
9,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,It's battery life is great. It's very responsi...,0.0


In [6]:
review.tail(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
413830,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,LOVE IT,0.0
413831,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,"Item was listed as new, but was not. When we t...",0.0
413832,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,4,good rugged phone that has a long-lasting batt...,0.0
413833,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,The phone is good,0.0
413834,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,used hard,0.0
413835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
413836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
413837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
413838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0
413839,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,4,Only downside is that apparently Verizon no lo...,0.0


In [7]:
#Check the shape of the data
review.shape
#400k reviews generated by Amazon users

(413840, 6)

Rustum's code to clean the data begins here. First remove the columns: 'Product Name' and 'Brand Name'. #TODO, dont

In [8]:
# Remove Product Name and Brand Name
data_dropped = review.drop(['Product Name','Brand Name'], axis = 1)

Remove rows which have null values in either Rating, Reviews, Review Votes, or Price

In [9]:
# Remove all data where either Rating, Reviews, Review Votes, or Price is null
cleaned_data = data_dropped.drop(data_dropped[data_dropped['Rating'].isnull() | data_dropped['Reviews'].isnull() | data_dropped['Review Votes'].isnull() | data_dropped['Price'].isnull()].index)
cleaned_data.head(10)

,Price,Rating,Reviews,Review Votes
0,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,199.99,5,Very pleased,0.0
3,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,199.99,4,Great phone to replace my lost phone. The only...,0.0
5,199.99,1,I already had a phone with problems... I know ...,1.0
6,199.99,2,The charging port was loose. I got that solder...,0.0
7,199.99,2,"Phone looks good but wouldn't stay charged, ha...",0.0
8,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0
9,199.99,3,It's battery life is great. It's very responsi...,0.0


Here I used the .translate function to remove punctuation from the reviews. Since our dataset is so large, str.replace would take too long and could potentially crash kernels. The translate function is a python function based in C and works very fast. I first join all the items with a character sequence that does not exist in the data: '|aa'. I then translate all the punctuations in the punctuations variable and finally split on this sequence with '|aa'.

In [10]:
punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~' 
translations = str.maketrans(dict.fromkeys(punctuations, ''))

cleaned_data['Reviews'] = '|aa'.join(cleaned_data['Reviews'].tolist()).translate(translations).split('|aa')

In [11]:
cleaned_data.head(10)

,Price,Rating,Reviews,Review Votes
0,199.99,5,I feel so LUCKY to have found this used phone ...,1.0
1,199.99,4,nice phone nice up grade from my pantach revue...,0.0
2,199.99,5,Very pleased,0.0
3,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,199.99,4,Great phone to replace my lost phone The only ...,0.0
5,199.99,1,I already had a phone with problems I know it ...,1.0
6,199.99,2,The charging port was loose I got that soldere...,0.0
7,199.99,2,Phone looks good but wouldnt stay charged had ...,0.0
8,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0
9,199.99,3,Its battery life is great Its very responsive ...,0.0


In [12]:
#This concludes Rustum's cleaning

In [13]:
#Ben's stopword (meaningless words) cleaning begin here
#begins by getting all review entries from all rows in the best ratings =5 and worst ratings =1 and putting into array
best = cleaned_data.loc[cleaned_data['Rating'] == 5]['Reviews'].str.cat(sep=' ')
worst = cleaned_data.loc[cleaned_data['Rating'] == 1]['Reviews'].str.cat(sep=' ')

In [14]:
#produces the frequencies of each individual word in both best and worst categories
bestlist = Counter(best.split()).most_common()
worstlist = Counter(worst.split()).most_common()

In [15]:
#this part iterates through the most first i(currently set at hundred) words of both best and worst reviews and selects words that are
#present in both. It stops after the most frequent i(100) words that are in both best and worst reviews have been found.
#It returns a list of these words which should suffice as a stop list since if the words can exist in both best and
#worst and are commonly found. They don't have bearing on the rating. Made into a function if you feel you need more
#stop words. 
def stop_words(i):
    common_words = []
    while len(common_words) < 100:
        common_words = []
        b = [x[0] for x in bestlist][:i]
        w = [x[0] for x in worstlist][:i]
        for x in range(i): 
            for y in range(i): 
                if b[x] == w[y]: 
                    common_words.append(b[x])
        i += 1
    return common_words
common_words = stop_words(100)
common_words

['the',
 'I',
 'and',
 'phone',
 'a',
 'it',
 'to',
 'is',
 'for',
 'this',
 'with',
 'my',
 'of',
 'in',
 'was',
 'that',
 'on',
 'great',
 'but',
 'you',
 'have',
 'as',
 'good',
 'The',
 'very',
 'not',
 'so',
 'It',
 'like',
 'had',
 'all',
 'one',
 'its',
 'use',
 'from',
 'has',
 'be',
 'an',
 'are',
 'This',
 'just',
 'can',
 'screen',
 'no',
 'new',
 'me',
 'at',
 'than',
 'or',
 'i',
 'battery',
 'product',
 'really',
 'get',
 'will',
 'phones',
 'camera',
 'about',
 'am',
 'time',
 'if',
 'up',
 'would',
 'more',
 'only',
 'out',
 'Im',
 'when',
 'what',
 'card',
 'any',
 'which',
 'your',
 'got',
 'bought',
 'buy',
 'do',
 'dont',
 'they',
 'other',
 'because',
 'used',
 'work',
 'came',
 'been',
 'now',
 'some',
 'also',
 'even',
 'Phone',
 'does',
 'there',
 'by',
 'Samsung',
 'back',
 '2',
 'purchase',
 'seller',
 'day',
 'first',
 'did']

In [16]:
# Ben: now we have a list of stop words which can be removed before moel processing

In [17]:
#extract the comments from the data
comments = review['Reviews']
comments

0         I feel so LUCKY to have found this used (phone...
1         nice phone, nice up grade from my pantach revu...
2                                              Very pleased
3         It works good but it goes slow sometimes but i...
4         Great phone to replace my lost phone. The only...
5         I already had a phone with problems... I know ...
6         The charging port was loose. I got that solder...
7         Phone looks good but wouldn't stay charged, ha...
8         I originally was using the Samsung S2 Galaxy f...
9         It's battery life is great. It's very responsi...
10        My fiance had this phone previously, but cause...
11        This is a great product it came after two days...
12        These guys are the best! I had a little situat...
13        I'm really disappointed about my phone and ser...
14        Ordered this phone as a replacement for the sa...
15        Had this phone before and loved it but was not...
16        I was able to get the phone I 

In [18]:
#check the data type of the reviews
type(comments[0])
#it is a string

str

In [19]:
#Now we create a list containing all the comments
comments_num = comments.shape[0]
comments_str = []

#filter the data so it only includes comments that are strings
#and make it into a list
for i in range(comments_num):
    if type(comments[i]) == str:
        comments_str.append(str(comments[i]))

In [20]:
#make all comments into one string
transformed_comment = ""
for reviews in comments_str:
    transformed_comment += reviews

In [21]:
#word count of all comments
len(transformed_comment)

89668420

In [22]:
import re

In [23]:
#Clean all punctuations and make all words into a list
comment_words = re.split(r'\W+', transformed_comment)

In [24]:
import urllib.request
import collections
import math
import os
import random
import zipfile
import datetime as dt

In [25]:
# now use a function which builds the dataset for us

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    
    #The first step is setting up a “counter” list, 
    #which will store the number of times a word is 
    #found within the data-set. 
    
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    
    """
    The next part of this function creates 
    a dictionary, called dictionary which is 
    populated by keys corresponding to each unique word. 
    The value assigned to each unique word key is simply 
    an increasing integer count of the size of the dictionary.  
    So, for instance, the most common word will receive the 
    value 1, the second most common the value 2, the third 
    most common word the value 3, and so on (the integer 0 
    is assigned to the ‘UNK’ words).   This step creates a 
    unique integer value for each word within the vocabulary 
    – accomplishing the second step of the process which was 
    defined above.
    
    """
    
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [26]:
#implement a data collection function

def collect_data(vocabulary_size=10000):
    #we will use our comment_words here as our vocabulary
    data, count, dictionary, reverse_dictionary = build_dataset(comment_words,
                                                                vocabulary_size)
    return data, count, dictionary, reverse_dictionary

In [27]:
data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

In [28]:
#now collect data
vocabulary_size = 10000
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocabulary_size)

In [29]:
#We will be using the skip-gram method instead of CBOW
batch_size = 128
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 2       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

In [30]:
# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

In [31]:
#Now we can start creating the TensorFlow Model for the data

graph = tf.Graph()

In [32]:
#Creating the TensorFlow graph

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    # Construct the variables for the softmax
    weights = tf.Variable(
        tf.truncated_normal([embedding_size, vocabulary_size],
                          stddev=1.0 / math.sqrt(embedding_size)))
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases
    
    # convert train_context to a one-hot format
    train_one_hot = tf.one_hot(train_context, vocabulary_size)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))
    
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [43]:
def run(graph, num_steps):
    with tf.Session(graph=graph).as_default() as session:
      # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
    final_embeddings = normalized_embeddings.eval()

In [46]:
with tf.Session(graph=graph).as_default() as session:
    num_steps = 100
    softmax_start_time = dt.datetime.now()
    run(graph, num_steps=num_steps)
    softmax_end_time = dt.datetime.now()
    print("Softmax method took {} minutes to run 100 iterations".format((softmax_end_time-softmax_start_time).total_seconds()))

Initialized
Average loss at step  0 :  9.41921615600586
Nearest to all: banged, Recommended, Whether, defined, Kit, Shop, Third, factura,
Nearest to are: cargador, preventing, fit, which, disappears, replies, Marketplace, describing,
Nearest to my: gloves, notable, messenger, managed, accessed, unnecessary, where, reporting,
Nearest to an: Vodafone, READ, payment, natural, punch, box, teach, delight,
Nearest to now: printing, subway, pods, ACTIVATE, normal, swap, versatile, block,
Nearest to to: Tempered, Sometimes, gigantic, Ill, hard, sold, doubts, horrible,
Nearest to 2: boost, voicemail, COLOR, usually, NEW, BlueTooth, A916, gadgets,
Nearest to what: eating, veces, their, kindly, smudge, high, provide, earphone,
Nearest to t: desire, Classic, volume, freezed, teach, Transfer, haha, failed,
Nearest to product: actively, Typing, OFF, g, valid, hype, OnePlus, ensure,
Nearest to because: examine, blend, handset, United, cosmetic, Ace, 9300, twenty,
Nearest to phone: elsewhere, can, scr

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: Variable/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable)]]

Caused by op 'Variable/read', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "/anaconda3/lib/python3.6/asyncio/base_events.py", line 422, in run_fo

In [ ]:
with graph.as_default():

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    # Add variable initializer.
    init = tf.global_variables_initializer()


In [ ]:
num_steps = 50000
nce_start_time = dt.datetime.now()
run(graph, num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} minutes to run 100 iterations".format((nce_end_time-nce_start_time).total_seconds()))


In [ ]:
#tokenizing words
#lematizing, preprocessing text
#